# ทดลองทำการแสดงเส้นทางสถานที่จากตำแหน่งของ user ไปตำแหน่งของสถานที่ user เลือกจะไป

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
from langchain_openai import ChatOpenAI
import requests
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import re

llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url= os.getenv("url"),  
    api_key= os.getenv("api_key"),  
    max_tokens=1000  
)

# User location and search radius
user_location = (13.7563, 100.5018)  # Example: Bangkok
radius = 5  # Radius in kilometers

In [ ]:
# สร้างโมเดล Pydantic สำหรับการจัดการข้อมูลคำสำคัญที่ได้จาก LLM
class SearchKeyword(BaseModel):
    keyword: str = Field(description="คำสำคัญที่ได้จากการประมวลผลคำถาม") # เพิ่มเรืองการ promt

# Clean keyword function
def clean_keyword(keyword: str) -> str:
    unwanted_words = ["แถวนี้", "ใกล้ฉัน", "ใกล้", "ช่วยหา"]
    for word in unwanted_words:
        keyword = keyword.replace(word, "")
    return re.sub(r'\s+', ' ', keyword).strip()

# Search function for Logdo Map API
def search_logdo_map_api(keyword, user_location, radius):
    base_url = "https://search.longdo.com/mapsearch/json/search?"
    params = {
        'keyword': keyword,
        'lon': user_location[1],
        'lat': user_location[0],
        'span': radius,
        'locale': 'th',
        'key': os.getenv("key_longdo")
    }
    response = requests.get(base_url, params=params)
    return response.json() if response.status_code == 200 else None

# Process user query
def process_user_query(user_query, user_location, radius):
    # Parse keyword from LLM
    parser = JsonOutputParser(pydantic_object=SearchKeyword)
    prompt = PromptTemplate(
        template="""\
## คุณมีหน้าที่กรองคำสำคัญจากคำขอผู้ใช้. # เพิ่มเรืองการ promt

# คำขอผู้ใช้: {user_query}

# Your response should be structured as follows:
{format_instructions}
""",
        input_variables=["user_query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    chain = prompt | llm | parser
    event = chain.invoke({"user_query": user_query, "user_location": user_location})

    if event:
        keyword = event.get('keyword', '')
        cleaned_keyword = clean_keyword(keyword)
        if cleaned_keyword:
            places_from_api = search_logdo_map_api(cleaned_keyword, user_location, radius)
            return cleaned_keyword, places_from_api
    return None, None


In [53]:
user_query = "บอกรายชื่อเซเว่นแถวนี้ให้หน่อย"
print(f"คำขอผู้ใช้: {user_query}")
print(f"ตำแหน่งผู้ใช้: {user_location}")
print(f"รัศมี: {radius} กิโลเมตร")

คำขอผู้ใช้: บอกรายชื่อเซเว่นแถวนี้ให้หน่อย
ตำแหน่งผู้ใช้: (13.7563, 100.5018)
รัศมี: 5 กิโลเมตร


In [57]:
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        for place in places_from_api["data"]:
            print(place.get("name", "ไม่ทราบชื่อ"))
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น
รายชื่อสถานที่:
เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
เซเว่นอิเลฟเว่น ถนนดินสอ (รหัสสาขา: 00117)
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น ราชดำเนิน (รหัสสาขา: 00216)
เซเว่นอีเลฟเว่น
เซเว่นอีเลฟเว่น ราชดำเนิน 3 (รหัสสาขา: 01102)
เซเว่นอิเลฟเว่น ตรอกมหรรณพาราม (รหัสสาขา: 15076)
เซเว่นอิเลฟเว่น สะพานวันชาติ (รหัสสาขา: 04217)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนราชดำเนินกลาง (รหัสสาขา: 10866)
เซเว่นอิเลฟเว่น Booth ตะนาว (รหัสสาขา: 00585)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น ถนนมหรรณพ (รหัสสาขา: 04249)
เซเว่นอิเลฟเว่น Booth ข้าวสาร (รหัสสาขา: 01355)
เซเว่นอีเลฟเว่น ตรอกข้าวสาร (รหัสสาขา: 01777)
เซเว่นอีเลฟเว่น
เซเว่นอิเลฟเว่น บางลำภู 4 (รหัสสาขา: 17222)


In [ ]:
import requests
import ipywidgets as widgets
from IPython.display import display

# ฟังก์ชันร้องขอข้อมูลสถานที่จาก API ค้นหาสถานที่
def get_places(query, api_key):
    url = "https://api.longdo.com/Search/json/search"  # ตัวอย่าง URL ของ API ค้นหาสถานที่
    params = {
        "keyword": query,
        "locale": "th",
        "key": api_key
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching places: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the places API: {e}")
        return None

# ฟังก์ชันร้องขอข้อมูลเส้นทางจาก API หาเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon, api_key):
    url = "https://api.longdo.com/RouteService/json/route/guide"
    params = {
        "flon": user_lon,
        "flat": user_lat,
        "tlon": dest_lon,
        "tlat": dest_lat,
        "mode": "t",
        "restrict": 0,
        "locale": "th",
        "key": api_key
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching route: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the route API: {e}")
        return None

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_data, user_location, api_key):
    if places_data and "data" in places_data:
        places = places_data["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูด
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon, api_key)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    print(route_info)  # ปรับแต่งการแสดงผลตามต้องการ
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ตัวอย่างการใช้งาน
api_key = "your_api_key"  # ใส่ API Key ที่ใช้งานได้
user_location = (13.7563, 100.5018)  # พิกัดเริ่มต้นของผู้ใช้
query = "เซเว่นอิเลฟเว่น"  # คำค้นหา

# เรียก API ค้นหาสถานที่
places_from_api = get_places(query, api_key)

# แสดง Dropdown และค้นหาเส้นทาง
display_places_list(places_from_api, user_location, api_key)


Error fetching places: 404
ไม่พบข้อมูลสถานที่ในผลลัพธ์


# ลองใช้ url ที่ค้นหาตัวอื่นได้ด้วยที่ไม่ได้กำหนดตรงๆ และทำการลองแยกการเรียกใช้ API 2 รอบ เป็นคนละ Cell จากการคาดการว่าถ้าเรียกใช้ ภายใน Cell เดียวกันอาจจะเป็นปัญหา

In [54]:
# Cell 1: ดึงข้อมูลสถานที่จาก API
def fetch_places():
    # จำลองการดึงข้อมูลจาก API หรือใช้ API จริง
    places_data = {
        "data": [
            {"name": "เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2", "lat": 13.757852, "lon": 100.502241},
            {"name": "ห้างสรรพสินค้าเซ็นทรัล", "lat": 13.746746, "lon": 100.536511},
            # เพิ่มข้อมูลสถานที่เพิ่มเติมที่นี่
        ]
    }
    return places_data

places_from_api = fetch_places()  # เรียกใช้ฟังก์ชันนี้ใน cell แรก


In [ ]:
# Cell 2: แสดง Dropdown และเรียกใช้ API เส้นทาง
import requests
import ipywidgets as widgets
from IPython.display import display

# ฟังก์ชันร้องขอข้อมูลเส้นทางจาก API
def get_route(user_location, dest_lat, dest_lon):
    start_lat, start_lon = user_location
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "start_lat": start_lat,
        "start_lon": start_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
        "mode": "t",
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_data, user_location):
    if places_data and "data" in places_data:
        places = places_data["data"]
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่: ',
            value=options[0][1] if options else None
        )

        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location, dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    print(route_info)  
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        dropdown.observe(on_dropdown_change, names='value')
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
display_places_list(places_from_api, user_location)


Dropdown(description='สถานที่: ', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2', (13.757852, 100.502241)), ('ห้าง…


คุณเลือกสถานที่: ห้างสรรพสินค้าเซ็นทรัล
ละติจูด: 13.746746, ลองจิจูด: 100.536511

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 400, 'message': 'Bad request'}}

คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2
ละติจูด: 13.757852, ลองจิจูด: 100.502241

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 400, 'message': 'Bad request'}}


In [56]:
print("Params ที่ส่ง:", params)


Params ที่ส่ง: {'flon': 100.5018, 'flat': 13.7563, 'tlon': 100.501, 'tlat': 13.7581, 'mode': 't', 'restrict': 0, 'locale': 'th', 'key': '7b6f8a4c53a57fa8315fbdcf5b108c83'}


In [ ]:
params = {
    "flon": 100.5018,  # ลองจิจูดจุดเริ่มต้น
    "flat": 13.7563,  # ละติจูดจุดเริ่มต้น
    "tlon": 100.501,  # ลองจิจูดจุดปลายทาง
    "tlat": 13.7581,  # ละติจูดจุดปลายทาง
    "mode": "t",  # โหมดการเดินทาง (ถ้ามี)
    "restrict": 0,
    "locale": "th",
    "key": os.getenv("key_longdo") # API key ของคุณ
}

# ส่งคำขอไปยัง API
url = "https://api.longdo.com/RouteService/json/route/guide"  # URL ของ API
response = requests.get(url, params=params)

# ตรวจสอบผลลัพธ์
print(response.status_code)  # ตรวจสอบสถานะ HTTP
print(response.text)  # ตรวจสอบข้อความแสดงข้อผิดพลาด


200
{"meta":{"status":200,"from":{"lon":100.5018,"lat":13.7563},"to":{"lon":100.501,"lat":13.7581},"config":"tnullth0"},"data":[{"guide":[{"turn":4,"name":"วงเวียนอนุสาวรีย์ประชาธิปไตย","distance":39,"interval":10},{"turn":2,"name":"ถนนราชดำเนินกลาง","distance":292,"interval":74},{"turn":1,"name":"ถนนตะนาว","distance":26,"interval":9},{"turn":1,"name":"ถนนราชดำเนินกลาง","distance":53,"interval":9},{"turn":0,"name":"ถนนเชื่อมต่อ","distance":54,"interval":54},{"turn":1,"name":"ซอยดำเนินกลางเหนือ","distance":149,"interval":86}],"fdistance":0.9496057,"tdistance":67.87412,"distance":613,"interval":242,"penalty":90,"id":31226520}]}


In [57]:
print(response.url)  # ดูว่า URL ที่ส่งเข้า API เหมือนกันหรือไม่

https://api.longdo.com/RouteService/json/route/guide?flon=100.5018&flat=13.7563&tlon=100.501&tlat=13.7581&mode=t&restrict=0&locale=th&key=7b6f8a4c53a57fa8315fbdcf5b108c83


---------------------------------------------------------------------

# จำลองเส้นทางการเดินทาง แบบสมมุติข้อมูล

In [14]:
import folium

# ฟังก์ชันสร้างแผนที่และวาดเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่เริ่มต้นที่ตำแหน่งเริ่มต้น
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # วาดเส้นทางจากข้อมูล API
    for route in route_data.get("data", []):
        path = [(guide["lat"], guide["lon"]) for guide in route.get("guide", [])]
        folium.PolyLine(path, color="blue", weight=5, opacity=0.7).add_to(route_map)

    return route_map

# ตัวอย่างข้อมูลเส้นทางจาก API
route_info = {
    "meta": {"status": 200},
    "data": [
        {
            "guide": [
                {"lat": 13.7563, "lon": 100.5018},  # จุดเริ่มต้น
                {"lat": 13.7568, "lon": 100.5020},
                {"lat": 13.7571, "lon": 100.5025},  # จุดปลายทาง
            ],
            "distance": 622,
        }
    ],
}

# พิกัดเริ่มต้นและปลายทาง
start_coords = (13.7563, 100.5018)
end_coords = (13.7581, 100.5025)

# สร้างแผนที่พร้อมเส้นทาง
map_with_route = create_map_with_route(route_info, start_coords, end_coords)

# แสดงแผนที่
# map_with_route.save("route_map.html")
# print("แผนที่ถูกบันทึกในไฟล์ route_map.html")


# ทำให้แสดงเส้นทางการเดินทาง

In [15]:
import folium
from IPython.display import display, HTML

# ฟังก์ชันสร้างแผนที่และวาดเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่เริ่มต้นที่ตำแหน่งเริ่มต้น
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # วาดเส้นทางจากข้อมูล API
    for route in route_data.get("data", []):
        # ดึงข้อมูลจุดเส้นทาง
        path = [(guide["lat"], guide["lon"]) for guide in route.get("guide", [])]
        
        # ตรวจสอบว่าจุดปลายทางอยู่ในเส้นทางหรือไม่
        if path and path[-1] != end_coords:
            path.append(end_coords)  # เพิ่มจุดปลายทางเสมอ

        # วาดเส้นทาง
        folium.PolyLine(path, color="blue", weight=5, opacity=0.7).add_to(route_map)

    return route_map

# ตัวอย่างข้อมูลเส้นทางจาก API
route_info = {
    "meta": {"status": 200},
    "data": [
        {
            "guide": [
                {"lat": 13.7563, "lon": 100.5018},  # จุดเริ่มต้น
                {"lat": 13.7568, "lon": 100.5020},
                {"lat": 13.7571, "lon": 100.5025},  # จุดก่อนปลายทาง
            ],
            "distance": 622,
        }
    ],
}

# พิกัดเริ่มต้นและปลายทาง
start_coords = (13.7563, 100.5018)
end_coords = (13.7581, 100.5025)  # จุดปลายทางที่แท้จริง

# สร้างแผนที่พร้อมเส้นทาง
map_with_route = create_map_with_route(route_info, start_coords, end_coords)

# แสดงผลใน Jupyter Notebook
map_with_route_html = map_with_route._repr_html_()
display(HTML(map_with_route_html))


In [ ]:
import requests
import ipywidgets as widgets
from IPython.display import display, HTML
import folium

# ฟังก์ชันร้องขอข้อมูลเส้นทางจาก API
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide"  # เปลี่ยน URL เป็นของจริง
    params = {
        "flon": user_lon,
        "flat": user_lat,
        "tlon": dest_lon,
        "tlat": dest_lat,
        "mode": "t",
        "restrict": "0",
        "locale": "th",
        "key": os.getenv("key_longdo") # ใส่ key ที่ถูกต้อง
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None

# ฟังก์ชันสร้างแผนที่และวาดเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # ตรวจสอบข้อมูลเส้นทาง
    if "data" not in route_data or not route_data["data"]:
        raise ValueError("No route data available")

    route_map = folium.Map(location=start_coords, zoom_start=13)

    for route in route_data["data"]:
        if "guide" in route and route["guide"]:
            path = [(guide["lat"], guide["lon"]) for guide in route["guide"] if "lat" in guide and "lon" in guide]
            if path:
                folium.PolyLine(path, color="blue", weight=5, opacity=0.7).add_to(route_map)
        else:
            print("No guide data in route")

    return route_map


# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_data, user_location):
    if places_data and "data" in places_data:
        places = places_data["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูด
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:'
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    print(route_info)  # ปรับแต่งการแสดงผลตามต้องการ

                    # สร้างแผนที่ใหม่พร้อมเส้นทาง
                    map_with_route = create_map_with_route(route_info, user_location, (dest_lat, dest_lon))
                    map_with_route_html = map_with_route._repr_html_()

                    # แสดงแผนที่ใน Jupyter
                    display(HTML(map_with_route_html))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# พิกัดเริ่มต้น (ตัวอย่างจากตำแหน่งของผู้ใช้)
user_location = (13.7563, 100.5018)

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
display_places_list(places_from_api, user_location)


ไม่พบข้อมูลสถานที่ในผลลัพธ์


In [30]:
import folium
from IPython.display import display, HTML

# ฟังก์ชันสร้างแผนที่และวาดเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่เริ่มต้นที่ตำแหน่งเริ่มต้น
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # วาดเส้นทางจากข้อมูล API
    for route in route_data.get("data", []):
        # ดึงข้อมูลจุดเส้นทาง
        path = [(guide["lat"], guide["lon"]) for guide in route.get("guide", [])]
        
        # ตรวจสอบว่าจุดปลายทางอยู่ในเส้นทางหรือไม่
        if path and path[-1] != end_coords:
            path.append(end_coords)  # เพิ่มจุดปลายทางเสมอ

        # วาดเส้นทาง
        folium.PolyLine(path, color="blue", weight=5, opacity=0.7).add_to(route_map)

    return route_map

# ตัวอย่างฟังก์ชันรับค่าจาก Cell แรกและแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    # สร้างแผนที่พร้อมเส้นทาง
    map_with_route = create_map_with_route(route_info, user_location, destination_coords)

    # แสดงผลใน Jupyter Notebook
    map_with_route_html = map_with_route._repr_html_()
    display(HTML(map_with_route_html))

# ฟังก์ชันที่รับค่าจาก Cell แรกและแสดงผล
def display_route_from_api(route_info, user_location, destination_coords):
    if route_info:
        print("\nข้อมูลเส้นทางที่ได้รับจาก API:")
        print(route_info)  # สามารถปรับแต่งการแสดงผลตามต้องการ
        display_route_map(route_info, user_location, destination_coords)
    else:
        print("ไม่สามารถดึงข้อมูลเส้นทางได้")

# # ตัวอย่างข้อมูลเส้นทางจาก API (สามารถรับค่าจาก Cell แรกได้)
# route_info = {
#     "meta": {"status": 200},
#     "data": [
#         {
#             "guide": [
#                 {"lat": 13.7563, "lon": 100.5018},  # จุดเริ่มต้น
#                 {"lat": 13.7568, "lon": 100.5020},
#                 {"lat": 13.7571, "lon": 100.5025},  # จุดก่อนปลายทาง
#             ],
#             "distance": 622,
#         }
#     ],
# }

# ตัวอย่างพิกัดของผู้ใช้และปลายทาง
user_location = (13.7563, 100.5018)
destination_coords = (13.7581, 100.5025)  # จุดปลายทางที่แท้จริง

# เรียกใช้ฟังก์ชันเพื่อแสดงแผนที่และเส้นทาง
display_route_from_api(route_info, user_location, destination_coords)



ข้อมูลเส้นทางที่ได้รับจาก API:
{'meta': {'status': 200}, 'data': [{'guide': [{'lat': 13.7563, 'lon': 100.5018}, {'lat': 13.7568, 'lon': 100.502}, {'lat': 13.7571, 'lon': 100.5025}], 'distance': 622}]}


In [9]:
import folium
from IPython.display import display, HTML

# ตัวอย่างข้อมูลเส้นทางจาก API
route_info = {
    'meta': {
        'status': 200,
        'from': {'lon': 100.5018, 'lat': 13.7563},
        'to': {'lon': 100.49801614, 'lat': 13.75678826},
        'config': 'tnullth0'
    },
    'data': [{
        'guide': [
            {'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 14},
            {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 135, 'interval': 136},
            {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 55, 'interval': 10},
            {'turn': 1, 'name': 'ซอยดำเนินกลางใต้', 'distance': 346, 'interval': 62},
            {'turn': 1, 'name': 'ตรอกสาเก', 'distance': 100, 'interval': 17},
            {'turn': 1, 'name': 'ถนนบูรณศาสตร์', 'distance': 105, 'interval': 40},
            {'turn': 1, 'name': 'ซอยดำเนินกลางใต้', 'distance': 97, 'interval': 19}
        ],
        'fdistance': 0.9496057,
        'tdistance': 7.2593346,
        'distance': 877,
        'interval': 298,
        'penalty': 105,
        'id': 31288407
    }]
}

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            # เพิ่มแต่ละจุดใน guide ลงในแผนที่
            for i, guide in enumerate(guides):
                # แสดงข้อมูล guide โดยไม่มีพิกัด
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("ข้อมูลเส้นทาง:")
    print(route_info)
    print("\nกำลังสร้างแผนที่...")

    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# เรียกใช้ฟังก์ชัน
from_coords = (route_info['meta']['from']['lat'], route_info['meta']['from']['lon'])
to_coords = (route_info['meta']['to']['lat'], route_info['meta']['to']['lon'])
display_route_map(route_info, from_coords, to_coords)


ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.49801614, 'lat': 13.75678826}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 14}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 135, 'interval': 136}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 55, 'interval': 10}, {'turn': 1, 'name': 'ซอยดำเนินกลางใต้', 'distance': 346, 'interval': 62}, {'turn': 1, 'name': 'ตรอกสาเก', 'distance': 100, 'interval': 17}, {'turn': 1, 'name': 'ถนนบูรณศาสตร์', 'distance': 105, 'interval': 40}, {'turn': 1, 'name': 'ซอยดำเนินกลางใต้', 'distance': 97, 'interval': 19}], 'fdistance': 0.9496057, 'tdistance': 7.2593346, 'distance': 877, 'interval': 298, 'penalty': 105, 'id': 31288407}]}

กำลังสร้างแผนที่...


In [10]:
import requests
import ipywidgets as widgets
import folium
from IPython.display import display, HTML

# ฟังก์ชันร้องขอข้อมูลเส้นทางจาก API
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?flon=100.5018&flat=13.7563&tlon=100.501083757432&tlat=13.7581357466596&mode=t&restrict=0&locale=th&key=7b6f8a4c53a57fa8315fbdcf5b108c83"
    params = {
        "start_lat": user_lat,
        "start_lon": user_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            # เพิ่มแต่ละจุดใน guide ลงในแผนที่
            for i, guide in enumerate(guides):
                # แสดงข้อมูล guide โดยไม่มีพิกัด
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("ข้อมูลเส้นทาง:")
    print(route_info)
    print("\nกำลังสร้างแผนที่...")

    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_data, user_location):
    if places_data and "data" in places_data:
        places = places_data["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูด
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    print(route_info)  # ปรับแต่งการแสดงผลตามต้องการ
                    
                    # แสดงแผนที่หลังจากได้รับข้อมูลเส้นทาง
                    display_route_map(route_info, user_location, selected_coords)
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ตัวอย่างข้อมูลสถานที่จาก API (ต้องแทนที่ด้วยข้อมูลจริง)
places_from_api = {
    "data": [
        {"name": "สถานที่ A", "lat": 13.7563, "lon": 100.5018},
        {"name": "สถานที่ B", "lat": 13.7581, "lon": 100.5010}
    ]
}

# กำหนดตำแหน่งของผู้ใช้ (ตัวอย่าง)
user_location = (13.7563, 100.5018)

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
display_places_list(places_from_api, user_location)


Dropdown(description='สถานที่:', options=(('สถานที่ A', (13.7563, 100.5018)), ('สถานที่ B', (13.7581, 100.501)…


คุณเลือกสถานที่: สถานที่ B
ละติจูด: 13.7581, ลองจิจูด: 100.501

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.501083757432, 'lat': 13.7581357466596}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 12}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 292, 'interval': 74}, {'turn': 1, 'name': 'ถนนตะนาว', 'distance': 26, 'interval': 12}, {'turn': 1, 'name': 'ถนนราชดำเนินกลาง', 'distance': 53, 'interval': 14}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 54, 'interval': 10}, {'turn': 1, 'name': 'ซอยดำเนินกลางเหนือ', 'distance': 158, 'interval': 28}], 'fdistance': 0.9496057, 'tdistance': 73.52046, 'distance': 622, 'interval': 150, 'penalty': 90, 'id': 31255967}]}
ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.501083757432, 'lat': 13.7581357466596}, 'config': 'tnullth0'}, 'dat


คุณเลือกสถานที่: สถานที่ A
ละติจูด: 13.7563, ลองจิจูด: 100.5018

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.501083757432, 'lat': 13.7581357466596}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 12}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 292, 'interval': 74}, {'turn': 1, 'name': 'ถนนตะนาว', 'distance': 26, 'interval': 12}, {'turn': 1, 'name': 'ถนนราชดำเนินกลาง', 'distance': 53, 'interval': 14}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 54, 'interval': 10}, {'turn': 1, 'name': 'ซอยดำเนินกลางเหนือ', 'distance': 158, 'interval': 28}], 'fdistance': 0.9496057, 'tdistance': 73.52046, 'distance': 622, 'interval': 150, 'penalty': 90, 'id': 25054144}]}
ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.501083757432, 'lat': 13.7581357466596}, 'config': 'tnullth0'}, 'da

# ปรับ Cell 5 ให้เก็บค่าที่กดใน drop drown ด้วย

In [17]:
import requests
import ipywidgets as widgets
from IPython.display import display

# ฟังก์ชันร้องขอข้อมูลเส้นทางจาก API
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"  # เปลี่ยน URL เป็นของจริง
    params = {
        "start_lat": user_lat,
        "start_lon": user_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        selected_coords = None  # ตัวแปรเก็บพิกัดที่เลือกจาก dropdown

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            nonlocal selected_coords  # ใช้ตัวแปร selected_coords ภายนอกฟังก์ชัน
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    print(route_info)  # ปรับแต่งการแสดงผลตามต้องการ
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)

        return selected_coords  # คืนค่าพิกัดที่เลือก

    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
        return None

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        selected_coords = display_places_list(places_from_api, user_location)  # เรียกฟังก์ชันเพื่อแสดง Dropdown และค้นหาเส้นทาง
        if selected_coords:
            print(f"\nพิกัดที่เลือก: {selected_coords}")
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: รายชื่อเซเว่น
รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(), value=None)

# Cell 6 ต่อการการดึงค่าขื่อสถานที่ออกมา

In [18]:
import folium
from IPython.display import display, HTML

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            # เพิ่มแต่ละจุดใน guide ลงในแผนที่
            for i, guide in enumerate(guides):
                # แสดงข้อมูล guide โดยไม่มีพิกัด
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("ข้อมูลเส้นทาง:")
    print(route_info)
    print("\nกำลังสร้างแผนที่...")

    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        selected_coords = None  # ตัวแปรเก็บพิกัดที่เลือกจาก dropdown

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            nonlocal selected_coords  # ใช้ตัวแปร selected_coords ภายนอกฟังก์ชัน
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))  # แสดงแผนที่ที่มีเส้นทาง
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)

        return selected_coords  # คืนค่าพิกัดที่เลือก

    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
        return None

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        selected_coords = display_places_list(places_from_api, user_location)  # เรียกฟังก์ชันเพื่อแสดง Dropdown และค้นหาเส้นทาง
        if selected_coords:
            print(f"\nพิกัดที่เลือก: {selected_coords}")
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น
รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
ละติจูด: 13.757852, ลองจิจูด: 100.502241

กำลังดึงข้อมูลเส้นทาง...
Error connecting to the API: Expecting value: line 1 column 1 (char 0)
ไม่สามารถดึงข้อมูลเส้นทางได้


In [19]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            # เพิ่มแต่ละจุดใน guide ลงในแผนที่
            for i, guide in enumerate(guides):
                # แสดงข้อมูล guide โดยไม่มีพิกัด
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("ข้อมูลเส้นทาง:")
    print(route_info)
    print("\nกำลังสร้างแผนที่...")

    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        selected_coords = None  # ตัวแปรเก็บพิกัดที่เลือกจาก dropdown

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            nonlocal selected_coords  # ใช้ตัวแปร selected_coords ภายนอกฟังก์ชัน
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))  # แสดงแผนที่ที่มีเส้นทาง
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)

        return selected_coords  # คืนค่าพิกัดที่เลือก

    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
        return None

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(start_lat, start_lon, end_lat, end_lon):
    try:
        # ใส่โค้ดในการดึงข้อมูลเส้นทางที่นี่
        # ถ้าสำเร็จให้ส่งข้อมูล route_info กลับไป
        # สมมติว่า return เป็นโครงสร้าง JSON ที่มีข้อมูลเกี่ยวกับเส้นทาง
        route_info = {
            "data": [{"guide": [{"name": "จุดที่ 1", "distance": 100}], "route": "เส้นทางหลัก"}]
        }
        return route_info
    except Exception as e:
        print(f"Error: {e}")
        return None

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        selected_coords = display_places_list(places_from_api, user_location)  # เรียกฟังก์ชันเพื่อแสดง Dropdown และค้นหาเส้นทาง
        if selected_coords:
            print(f"\nพิกัดที่เลือก: {selected_coords}")
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น
รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น Booth ข้าวสาร (รหัสสาขา: 01355)
ละติจูด: 13.7583049, ลองจิจูด: 100.49830399

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
ข้อมูลเส้นทาง:
{'data': [{'guide': [{'name': 'จุดที่ 1', 'distance': 100}], 'route': 'เส้นทางหลัก'}]}

กำลังสร้างแผนที่...


In [20]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    # สร้างแผนที่
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            # เพิ่มแต่ละจุดใน guide ลงในแผนที่
            for i, guide in enumerate(guides):
                # แสดงข้อมูล guide โดยไม่มีพิกัด
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    # print("ข้อมูลเส้นทาง:")
    print(route_info)
    print("\nกำลังสร้างแผนที่...")

    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        selected_coords = None  # ตัวแปรเก็บพิกัดที่เลือกจาก dropdown

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            nonlocal selected_coords  # ใช้ตัวแปร selected_coords ภายนอกฟังก์ชัน
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    print("\nข้อมูลเส้นทาง:")
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))  # แสดงแผนที่ที่มีเส้นทาง
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)

        return selected_coords  # คืนค่าพิกัดที่เลือก

    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
        return None

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?flon=100.5018&flat=13.7563&tlon=100.501083757432&tlat=13.7581357466596&mode=t&restrict=0&locale=th&key=7b6f8a4c53a57fa8315fbdcf5b108c83"  # เปลี่ยน URL เป็นของจริง
    params = {
        "start_lat": user_lat,
        "start_lon": user_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None

# เรียกฟังก์ชันแสดง Dropdown และค้นหาเส้นทาง
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("รายชื่อสถานที่:")
        selected_coords = display_places_list(places_from_api, user_location)  # เรียกฟังก์ชันเพื่อแสดง Dropdown และค้นหาเส้นทาง
        if selected_coords:
            print(f"\nพิกัดที่เลือก: {selected_coords}")
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("ไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น, รายชื่อ,
รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(), value=None)

# ปรับโค๊ด ค่าที่ส่งไปยังฟังก์ชัน get_routeไม่ได้ถูกตั้งค่าใหม่หลังจากผู้ใช้เลือกใน Dropdown

In [ ]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                # แก้ไขตรงนี้เพื่อให้เรียกฟังก์ชัน get_route ด้วยพารามิเตอร์ที่ถูกต้อง
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")


# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"  # URL API จริง
    params = {
        "start_lat": user_lat,
        "start_lon": user_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
        "mode": "t",
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")  # ใช้ API Key จริง
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            print("\nผลลัพธ์จาก API:")
            print(response.json())
            return response.json()
        else:
            print(f"Error: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None


# ฟังก์ชันประมวลผลคำถามผู้ใช้และค้นหาสถานที่
# (สมมติว่ามีฟังก์ชัน `process_user_query` อยู่แล้ว)
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ถนนมหรรณพ (รหัสสาขา: 04249)
ละติจูด: 13.753872, ลองจิจูด: 100.498801

กำลังดึงข้อมูลเส้นทาง...

ผลลัพธ์จาก API:
{'meta': {'status': 400, 'message': 'Bad request'}}

ข้อมูลเส้นทาง:
{'meta': {'status': 400, 'message': 'Bad request'}}

กำลังสร้างแผนที่...


In [ ]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                # เรียกใช้ฟังก์ชัน get_route โดยใช้พิกัดที่เลือก
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"  # URL API จริง
    params = {
        "start_lat": user_lat,
        "start_lon": user_lon,
        "end_lat": dest_lat,
        "end_lon": dest_lon,
        "mode": "t",
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")  # ใช้ API Key จริง
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            print("\nผลลัพธ์จาก API:")
            print(response.json())
            return response.json()
        else:
            print(f"Error: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"Error connecting to the API: {e}")
        return None



# ฟังก์ชันประมวลผลคำถามผู้ใช้และค้นหาสถานที่
# (สมมติว่ามีฟังก์ชัน `process_user_query` อยู่แล้ว)
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอีเลฟเว่น ตรอกข้าวสาร (รหัสสาขา: 01777)
ละติจูด: 13.7585479999, ลองจิจูด: 100.498331

กำลังดึงข้อมูลเส้นทาง...

ผลลัพธ์จาก API:
{'meta': {'status': 400, 'message': 'Bad request'}}

ข้อมูลเส้นทาง:
{'meta': {'status': 400, 'message': 'Bad request'}}

กำลังสร้างแผนที่...


# อันนี้ใช้ได้

In [ ]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {dropdown.label}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                # เรียกใช้ฟังก์ชัน get_route โดยใช้พิกัดที่เลือก
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "flon": user_lon,  # ลองจิจูดของผู้ใช้
        "flat": user_lat,  # ละติจูดของผู้ใช้
        "tlon": dest_lon,  # ลองจิจูดของปลายทาง
        "tlat": dest_lat,  # ละติจูดของปลายทาง
        "mode": "t",  # โมเดลการเดินทาง
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")  # ใช้ API Key จริง
    }

    try:
        response = requests.get(url, params=params)
        
        # ตรวจสอบว่า API ส่งสถานะ 200 (OK) หรือไม่
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data
            else:
                print(f"ข้อมูลเส้นทางไม่พบในผลลัพธ์: {data}")
                return None
        else:
            print(f"เกิดข้อผิดพลาดจาก API: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ API: {e}")
        return None



# ฟังก์ชันประมวลผลคำถามผู้ใช้และค้นหาสถานที่
# (สมมติว่ามีฟังก์ชัน `process_user_query` อยู่แล้ว)
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอีเลฟเว่น ตรอกข้าวสาร (รหัสสาขา: 01777)
ละติจูด: 13.7585479999, ลองจิจูด: 100.498331

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.498331, 'lat': 13.7585479999}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 111, 'interval': 66}, {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 280, 'interval': 98}, {'turn': 0, 'name': 'ถนนพระสุเมรุ', 'distance': 545, 'interval': 63}, {'turn': 0, 'name': 'ถนนจักรพงษ์', 'distance': 397, 'interval': 99}, {'turn': 0, 'name': 'ถนนข้าวสาร', 'distance': 318, 'interval': 177}], 'fdistance': 0.9496057, 'tdistance': 13.058486, 'distance': 1651, 'interval': 503, 'penalty': 75, 'id': 31508780}]}

กำลังสร้างแผนที่...



คุณเลือกสถานที่: เซเว่นอีเลฟเว่น ราชดำเนิน 3 (รหัสสาขา: 01102)
ละติจูด: 13.7574440002, ลองจิจูด: 100.500072

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.500072, 'lat': 13.7574440002}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 39, 'interval': 11}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 292, 'interval': 87}, {'turn': 1, 'name': 'ถนนตะนาว', 'distance': 26, 'interval': 14}, {'turn': 1, 'name': 'ถนนราชดำเนินกลาง', 'distance': 53, 'interval': 12}, {'turn': 0, 'name': 'ถนนเชื่อมต่อ', 'distance': 54, 'interval': 10}, {'turn': 1, 'name': 'ซอยดำเนินกลางเหนือ', 'distance': 64, 'interval': 12}], 'fdistance': 0.9496057, 'tdistance': 23.368593, 'distance': 528, 'interval': 146, 'penalty': 90, 'id': 31508801}]}

กำลังสร้างแผนที่...


# ลองใช้ LLM ทำการอธิบายข้อมูลเส้นทางที่ได้มาเป็นภาษาคน

In [24]:
# ฟังก์ชันแปลงข้อมูลเส้นทางเป็นข้อความเพื่อส่งให้ LLM
def generate_human_readable_directions(route_info, destination_name):
    if not route_info or "data" not in route_info:
        print("ไม่มีข้อมูลเส้นทางที่สามารถแปลงได้")
        return None

    # ข้อมูลจาก JSON
    guides = route_info["data"][0].get("guide", [])
    total_distance = route_info["data"][0].get("distance", "ไม่ทราบระยะทาง")
    total_time = route_info["data"][0].get("interval", "ไม่ทราบเวลา")
    start_coords = route_info["meta"]["from"]
    end_coords = route_info["meta"]["to"]

    # สร้างคำอธิบายเส้นทาง
    directions = f"เส้นทางจากตำแหน่งเริ่มต้น (ละติจูด: {start_coords['lat']}, ลองจิจูด: {start_coords['lon']})\n"
    directions += f"ไปยัง {destination_name} (ละติจูด: {end_coords['lat']}, ลองจิจูด: {end_coords['lon']})\n"
    directions += f"ระยะทางทั้งหมดประมาณ {total_distance} เมตร และใช้เวลาประมาณ {total_time} วินาที\n\n"
    directions += "รายละเอียดเส้นทาง:\n"

    for i, guide in enumerate(guides, start=1):
        turn_direction = guide.get("turn", "ไม่ระบุทิศทาง")
        place_name = guide.get("name", "ไม่ทราบชื่อสถานที่")
        distance = guide.get("distance", "ไม่ทราบระยะทาง")
        interval = guide.get("interval", "ไม่ทราบเวลา")
        directions += (
            f"{i}. เลี้ยว {turn_direction} ไปยัง {place_name} "
            f"(ระยะทาง: {distance} เมตร, ใช้เวลา: {interval} วินาที)\n"
        )

    # สร้าง prompt สำหรับ LLM
    prompt = f"""
    ข้อมูลเส้นทางที่ได้:
    
    {directions}
    
    กรุณาแปลงข้อมูลนี้ให้อยู่ในรูปแบบข้อความอธิบายเส้นทางที่เข้าใจง่าย โดยใช้ภาษาคน
    """
    return prompt

# การใช้งาน LLM
def explain_route_with_llm(route_info, destination_name):
    # สร้าง prompt
    prompt = generate_human_readable_directions(route_info, destination_name)
    if not prompt:
        print("ไม่สามารถสร้าง prompt ได้")
        return

    # ส่ง prompt ไปยัง LLM
    try:
        response = llm.invoke(prompt)  # ปรับตาม API หรือไลบรารีที่คุณใช้
        print("\nคำอธิบายเส้นทางจาก LLM:")
        print(response.content.strip())
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ LLM: {str(e)}")


In [25]:
# ข้อมูลตัวอย่างจาก API
route_info = {
    'meta': {
        'status': 200,
        'from': {'lon': 100.5018, 'lat': 13.7563},
        'to': {'lon': 100.501247, 'lat': 13.754661},
        'config': 'tnullth0'
    },
    'data': [{
        'guide': [
            {'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 236, 'interval': 70},
            {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 207, 'interval': 86}
        ],
        'fdistance': 0.9496057,
        'tdistance': 15.329851,
        'distance': 443,
        'interval': 156,
        'penalty': 30,
        'id': 31510244
    }]
}
destination_name = "เซเว่นอิเลฟเว่น ถนนดินสอ (รหัสสาขา: 00117)"

# เรียกฟังก์ชัน
explain_route_with_llm(route_info, destination_name)



คำอธิบายเส้นทางจาก LLM:
เริ่มต้นจากตำแหน่งของคุณที่ละติจูด 13.7563 และลองจิจูด 100.5018 ให้คุณเดินไปทางวงเวียนอนุสาวรีย์ประชาธิปไตย โดยให้เลี้ยวขวา (เลี้ยว 4) ซึ่งจะใช้ระยะทางประมาณ 236 เมตร และใช้เวลาประมาณ 70 วินาทีในการเดินทางถึงวงเวียนนี้ 

หลังจากนั้นให้คุณเลี้ยวซ้าย (เลี้ยว 2) เข้าสู่ถนนดินสอ แล้วเดินต่อไปอีกประมาณ 207 เมตร ซึ่งจะใช้เวลาประมาณ 86 วินาที 

เมื่อรวมทั้งหมดแล้ว คุณจะใช้เวลาทั้งสิ้นประมาณ 156 วินาที หรือราว ๆ 2 นาที ในการเดินทางไปยังเซเว่นอิเลฟเว่น ถนนดินสอ สาขา 00117 ที่ตั้งอยู่ที่ละติจูด 13.754661 และลองจิจูด 100.501247 โดยระยะทางรวมประมาณ 443 เมตร 

เดินทางปลอดภัยนะ!


ลองเรียงลำดับใหม่


In [42]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests


In [ ]:
# ตัวแปร global สำหรับเก็บค่าที่เลือก
selected_place_name = None  # ชื่อสถานที่ที่เลือก
selected_coords = None  # พิกัดของสถานที่ที่เลือก
current_route_info = None  # ข้อมูลเส้นทางล่าสุด

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    start_coords,  # ใช้จุดเริ่มต้นเพื่อแทนที่ตำแหน่งจริง (ถ้าพิกัดไม่มี)
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    global selected_place_name, selected_coords, current_route_info

    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            global selected_place_name, selected_coords, current_route_info

            selected_coords = change.new
            selected_place_name = next(
                (name for name, coords in options if coords == selected_coords), "ไม่ทราบชื่อ"
            )

            if selected_coords:
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {selected_place_name}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                current_route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if current_route_info:
                    display_route_map(current_route_info, user_location, (dest_lat, dest_lon))
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        # ตั้งค่าการทำงานเมื่อมีการเลือกสถานที่
        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")

# ฟังก์ชันที่ดึงข้อมูลเส้นทาง
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "flon": user_lon,  # ลองจิจูดของผู้ใช้
        "flat": user_lat,  # ละติจูดของผู้ใช้
        "tlon": dest_lon,  # ลองจิจูดของปลายทาง
        "tlat": dest_lat,  # ละติจูดของปลายทาง
        "mode": "t",  # โมเดลการเดินทาง
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")  # ใช้ API Key จริง
    }

    try:
        response = requests.get(url, params=params)
        
        # ตรวจสอบว่า API ส่งสถานะ 200 (OK) หรือไม่
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data
            else:
                print(f"ข้อมูลเส้นทางไม่พบในผลลัพธ์: {data}")
                return None
        else:
            print(f"เกิดข้อผิดพลาดจาก API: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ API: {e}")
        return None


In [44]:
# ฟังก์ชันสร้างคำอธิบายเส้นทาง
def generate_human_readable_directions(route_data):
    directions = []
    for guide in route_data.get("data", [{}]).get("guide", []):
        step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
        directions.append(step)
    return "\n".join(directions)

# ฟังก์ชันแสดงคำอธิบายเส้นทางด้วย LLM
def explain_route_with_llm(route_data, destination_name):
    if not route_data:
        print("ไม่มีข้อมูลเส้นทาง")
        return

    print(f"\nเส้นทางไปยัง {destination_name}:")
    human_readable_directions = generate_human_readable_directions(route_data)
    print(human_readable_directions)


In [26]:
# จำลองค่าผู้ใช้
user_location = (14.022788, 99.978337)  # ตัวอย่างตำแหน่งผู้ใช้
user_query = "หาร้านอาหารใกล้ฉัน"  # ตัวอย่างคำถามผู้ใช้
radius = 5000  # รัศมีค้นหา

# สมมติว่ามีฟังก์ชัน `process_user_query` ที่คืนค่า keyword และข้อมูลสถานที่
keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        display_places_list(places_from_api, user_location)

        # เรียกฟังก์ชันสร้างคำอธิบายเส้นทาง เมื่อเลือกสถานที่แล้ว
        if current_route_info and selected_place_name:
            explain_route_with_llm(current_route_info, selected_place_name)
        else:
            print("กรุณาเลือกสถานที่จาก Dropdown ก่อน")
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")


คำค้นหา: ร้านอาหาร

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('ร้านป้าสำราญ แคนตั๊กกี้', (14.022909, 99.981014)), ('ร้านอาหารเค.ย…

NameError: name 'current_route_info' is not defined


คุณเลือกสถานที่: เท็กซัส สเต็ก เค ยู บีฟ  
ละติจูด: 14.025270694332686, ลองจิจูด: 99.96986208954628

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 99.978337, 'lat': 14.022788}, 'to': {'lon': 99.96986208954628, 'lat': 14.025270694332686}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'ถนนเชื่อมต่อ', 'distance': 1025, 'interval': 178}], 'fdistance': 5.064674, 'tdistance': 84.07755, 'distance': 1025, 'interval': 178, 'penalty': 15, 'id': 39861417}]}

กำลังสร้างแผนที่...


---------------------------------------------------------------------------------------

In [48]:
import folium
from IPython.display import display, HTML
import ipywidgets as widgets
import requests
import openai  # ต้องติดตั้ง openai-py ก่อนใช้

# ฟังก์ชันสร้างแผนที่พร้อมเส้นทาง
def create_map_with_route(route_data, start_coords, end_coords):
    route_map = folium.Map(location=start_coords, zoom_start=15)

    # เพิ่มจุดเริ่มต้นและปลายทาง
    folium.Marker(start_coords, popup="จุดเริ่มต้น", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(end_coords, popup="จุดปลายทาง", icon=folium.Icon(color="red")).add_to(route_map)

    # ตรวจสอบว่ามีข้อมูล guide หรือไม่
    for route in route_data.get("data", []):
        guides = route.get("guide", [])
        if guides:
            for i, guide in enumerate(guides):
                folium.Marker(
                    [guide.get("lat", start_coords[0]), guide.get("lon", start_coords[1])],
                    popup=f"จุดที่ {i+1}: {guide['name']} ({guide['distance']} เมตร)",
                    icon=folium.Icon(color="blue", icon="info-sign")
                ).add_to(route_map)

    # วาดเส้นระหว่างจุดเริ่มต้นและปลายทาง
    folium.PolyLine([start_coords, end_coords], color="blue", weight=5, opacity=0.7).add_to(route_map)
    return route_map

# ฟังก์ชันแสดงแผนที่
def display_route_map(route_info, user_location, destination_coords):
    print("\nข้อมูลเส้นทาง:")
    print(route_info)

    print("\nกำลังสร้างแผนที่...")
    route_map = create_map_with_route(route_info, user_location, destination_coords)
    route_map_html = route_map._repr_html_()
    display(HTML(route_map_html))


In [ ]:
# ฟังก์ชันที่ดึงข้อมูลเส้นทางจาก API
def get_route(user_lat, user_lon, dest_lat, dest_lon):
    url = "https://api.longdo.com/RouteService/json/route/guide?"
    params = {
        "flon": user_lon,
        "flat": user_lat,
        "tlon": dest_lon,
        "tlat": dest_lat,
        "mode": "t",
        "restrict": 0,
        "locale": "th",
        "key": os.getenv("key_longdo")
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data
            else:
                print(f"ข้อมูลเส้นทางไม่พบในผลลัพธ์: {data}")
                return None
        else:
            print(f"เกิดข้อผิดพลาดจาก API: {response.status_code}, Message: {response.text}")
            return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ API: {e}")
        return None


In [55]:
# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:',
            value=options[0][1] if options else None
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                # ค้นหาชื่อสถานที่จากพิกัดที่เลือก
                selected_place_name = next((place[0] for place in options if place[1] == selected_coords), "ไม่ทราบชื่อ")
                dest_lat, dest_lon = selected_coords
                print(f"\nคุณเลือกสถานที่: {selected_place_name}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                    explain_route_with_llm(route_info, selected_place_name)
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")


In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# ฟังก์ชันสร้างคำอธิบายเส้นทาง
def generate_human_readable_directions(route_data):
    directions = []
    # ตรวจสอบว่า route_data เป็น list หรือ dict
    if isinstance(route_data, list):
        for route in route_data:
            guides = route.get("guide", [])
            for guide in guides:
                step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
                directions.append(step)
    elif isinstance(route_data, dict):
        guides = route_data.get("guide", [])
        for guide in guides:
            step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
            directions.append(step)
    return "\n".join(directions)

# ฟังก์ชันแสดงคำอธิบายเส้นทางด้วย LLM
def explain_route_with_llm(route_data, destination_name, llm):
    if not route_data:
        print("ไม่มีข้อมูลเส้นทาง")
        return

    human_readable_directions = generate_human_readable_directions(route_data)
    prompt = f"""
คุณเป็นผู้ช่วย AI ที่เชี่ยวชาญด้านการอธิบายเส้นทาง
โปรดอธิบายเส้นทางไปยัง "{destination_name}" ด้วยข้อความที่เข้าใจง่าย:

ข้อมูลเส้นทาง:
{human_readable_directions}
"""
    print(human_readable_directions)
    print(destination_name)
    print(route_data)
    print(llm)
    try:
        
        # ใช้ ChatOpenAI API สำหรับการส่ง prompt และรับคำตอบ
        response = llm.invoke([{"role": "user", "content": prompt}])  # ใช้ invoke แทน predict_messages

        # ตรวจสอบว่าผลลัพธ์มีค่าและไม่เป็น None
        if response:
            # เข้าถึงข้อความของผลลัพธ์
            explanation = response[0].content.strip()  # เข้าถึงเนื้อหาผลลัพธ์จาก AIMessage
            print(f"\nคำอธิบายเส้นทางไปยัง {destination_name}:\n{explanation}")
        else:
            print("ไม่พบคำตอบจาก LLM.")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเรียกใช้ LLM: {e}")



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# ฟังก์ชันสร้างคำอธิบายเส้นทาง
def generate_human_readable_directions(route_data):
    directions = []
    # ตรวจสอบว่า route_data เป็น list หรือ dict
    if isinstance(route_data, list):
        for route in route_data:
            guides = route.get("guide", [])
            for guide in guides:
                step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
                directions.append(step)
    elif isinstance(route_data, dict):
        guides = route_data.get("guide", [])
        for guide in guides:
            step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
            directions.append(step)
    return "\n".join(directions)

# ฟังก์ชันแสดงคำอธิบายเส้นทางด้วย LLM
def explain_route_with_llm(route_data, destination_name, llm):
    if not route_data:
        print("ไม่มีข้อมูลเส้นทาง")
        return

    human_readable_directions = generate_human_readable_directions(route_data)
    prompt = f"""
คุณเป็นผู้ช่วย AI ที่เชี่ยวชาญด้านการอธิบายเส้นทาง
โปรดอธิบายเส้นทางไปยัง "{destination_name}" ด้วยข้อความที่เข้าใจง่าย:

ข้อมูลเส้นทาง:
{human_readable_directions}
"""
    print(human_readable_directions)
    print(destination_name)
    print(route_data)
    print(llm)
 
        
    # ใช้ ChatOpenAI API สำหรับการส่ง prompt และรับคำตอบ
    https://api.longdo.com/map/services/address?lon=100.53726&lat=13.72427&noelevation=1&key=[YOUR-KEY-API]

    response = llm.invoke([SystemMessage(content=prompt), HumanMessage(content="ผมอยู่ รัชดา")])  # ใช้ invoke แทน predict_messages
    print(response)
    # ตรวจสอบว่าผลลัพธ์มีค่าและไม่เป็น None
    if response:
          print(f"\nคำอธิบายเส้นทางไปยัง {destination_name}:\n{response.content}")
    else:
        print("ไม่พบคำตอบจาก LLM.")
    

keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        # เรียกใช้ฟังก์ชันแสดง Dropdown
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:'
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                selected_place_name = dropdown.label  # เก็บชื่อสถานที่
                print(f"\nคุณเลือกสถานที่: {selected_place_name}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                    explain_route_with_llm(route_info, selected_place_name, llm)  # เรียกใช้ LLM เพื่ออธิบายเส้นทาง
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")


SyntaxError: invalid syntax (1295986332.py, line 42)

# มาทำตรงนี้ เรื่องแนะนำเส้นทาง

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# ฟังก์ชันสร้างคำอธิบายเส้นทาง
def generate_human_readable_directions(route_data):
    directions = []
    # ตรวจสอบว่า route_data เป็น list หรือ dict
    if isinstance(route_data, list):
        for route in route_data:
            guides = route.get("guide", [])
            for guide in guides:
                step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
                directions.append(step)
    elif isinstance(route_data, dict):
        guides = route_data.get("guide", [])
        for guide in guides:
            step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
            directions.append(step)
    return "\n".join(directions)

# ฟังก์ชันแสดงคำอธิบายเส้นทางด้วย LLM
def explain_route_with_llm(route_data, destination_name, llm):
    if not route_data:
        print("ไม่มีข้อมูลเส้นทาง")
        return

    human_readable_directions = generate_human_readable_directions(route_data)
    prompt = f"""
คุณเป็นผู้ช่วย AI ที่เชี่ยวชาญด้านการอธิบายเส้นทาง
โปรดอธิบายเส้นทางไปยัง "{destination_name}" ด้วยข้อความที่เข้าใจง่าย:

ข้อมูลเส้นทาง:
{human_readable_directions}
"""
    print(human_readable_directions)
    print(destination_name)
    print(route_data)
    print(llm)
    
    response = llm.invoke([SystemMessage(content=prompt), HumanMessage(content="")])  # ใช้ invoke แทน predict_messages
    print(response)
    # ตรวจสอบว่าผลลัพธ์มีค่าและไม่เป็น None
    if response:
          print(f"\nคำอธิบายเส้นทางไปยัง {destination_name}:\n{response.content}")
    else:
        print("ไม่พบคำตอบจาก LLM.")
    

keyword, places_from_api = process_user_query(user_query, user_location, radius)

if keyword:
    print(f"คำค้นหา: {keyword}")
    if places_from_api and "data" in places_from_api:
        print("\nรายชื่อสถานที่:")
        # เรียกใช้ฟังก์ชันแสดง Dropdown
        display_places_list(places_from_api, user_location)
    else:
        print("\nไม่พบข้อมูลสถานที่ในผลลัพธ์")
else:
    print("\nไม่พบคำสำคัญจากการประมวลผล")

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]

        # สร้างตัวเลือก Dropdown พร้อมพิกัดละติจูดและลองจิจูดจากสถานที่
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:'
        )

        # ฟังก์ชันเมื่อเปลี่ยนค่าของ Dropdown
        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                selected_place_name = dropdown.label  # เก็บชื่อสถานที่
                print(f"\nคุณเลือกสถานที่: {selected_place_name}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")

                # ดึงข้อมูลเส้นทาง
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                    explain_route_with_llm(route_info, selected_place_name, llm)  # เรียกใช้ LLM เพื่ออธิบายเส้นทาง
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        dropdown.observe(on_dropdown_change, names='value')

        # แสดง Dropdown
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")


คำค้นหา: เซเว่น

รายชื่อสถานที่:


Dropdown(description='สถานที่:', options=(('เซเว่นอิเลฟเว่น ถนนดินสอ จุด 3 (รหัสสาขา: 13597)', (13.75523416, 1…


คุณเลือกสถานที่: เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
ละติจูด: 13.757852, ลองจิจูด: 100.502241

กำลังดึงข้อมูลเส้นทาง...

ข้อมูลเส้นทาง:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.502241, 'lat': 13.757852}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 111, 'interval': 66}, {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 106, 'interval': 9}], 'fdistance': 0.9496057, 'tdistance': 10.145972, 'distance': 217, 'interval': 75, 'penalty': 30, 'id': 31510391}]}

กำลังสร้างแผนที่...



เซเว่นอิเลฟเว่น ถนนดินสอ จุด 2 (รหัสสาขา: 09797)
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 100.502241, 'lat': 13.757852}, 'config': 'tnullth0'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 111, 'interval': 66}, {'turn': 2, 'name': 'ถนนดินสอ', 'distance': 106, 'interval': 9}], 'fdistance': 0.9496057, 'tdistance': 10.145972, 'distance': 217, 'interval': 75, 'penalty': 30, 'id': 31510391}]}
client=<openai.resources.chat.completions.Completions object at 0x000001C4F671AA20> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C4F65180E0> root_client=<openai.OpenAI object at 0x000001C4F658FAD0> root_async_client=<openai.AsyncOpenAI object at 0x000001C4F651BFE0> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='http://111.223.37.52/v1' max_tokens=1000
content='ดูเหมือนว่าคุณยังไม่ได้ระบุจุดเริ่มต้นของเส้นทาง หากคุณสามารถบอกจุดเริ่มต้นหร

https://api.longdo.com/map/services/address?

In [61]:
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import requests

# ข้อมูลการตั้งค่า
longdo_api_key = "<ใส่คีย์ของคุณที่นี่>"
llm_api_key = "<ใส่คีย์ของคุณที่นี่>"

# ตั้งค่า ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=llm_api_key)

lat, lon = user_location
def get_location_name(lat, lon, api_key):
    url = "https://api.longdo.com/map/services/address"
    params = {
        "lon": lon,
        "lat": lat,
        "key": api_key
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200 and response.headers.get("Content-Type") == "application/json":
            data = response.json()
            return data.get("address", {}).get("address", "ไม่พบชื่อสถานที่")
        else:
            print(f"Longdo API ส่งผลลัพธ์ที่ไม่ใช่ JSON: {response.text}")
            return "ไม่สามารถดึงข้อมูลสถานที่ได้"
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเรียก Longdo API: {e}")
        return "ไม่สามารถดึงข้อมูลสถานที่ได้"

# ฟังก์ชันสร้างคำอธิบายเส้นทาง
def generate_human_readable_directions(route_data):
    directions = []
    if isinstance(route_data, list):
        for route in route_data:
            guides = route.get("guide", [])
            for guide in guides:
                step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
                directions.append(step)
    elif isinstance(route_data, dict):
        guides = route_data.get("guide", [])
        for guide in guides:
            step = f"{guide.get('name', 'ไม่ทราบจุด')} - ระยะทาง {guide.get('distance', 0)} เมตร"
            directions.append(step)
    return "\n".join(directions)

def explain_route_with_llm(route_data, destination_name, llm, user_lat, user_lon, longdo_api_key):
    if not route_data:
        print("ไม่มีข้อมูลเส้นทาง")
        return

    user_location_name = get_location_name(user_lat, user_lon, longdo_api_key)
    human_readable_directions = generate_human_readable_directions(route_data)
    prompt = f"""
คุณเป็นผู้ช่วย AI ที่เชี่ยวชาญด้านการอธิบายเส้นทาง
โปรดอธิบายเส้นทางจาก "{user_location_name}" ไปยัง "{destination_name}" ด้วยข้อความที่เข้าใจง่าย:

ข้อมูลเส้นทาง:
{human_readable_directions}
""".encode('utf-8').decode('utf-8')  # จัดการข้อความ Unicode

    try:
        response = llm.invoke([
            SystemMessage(content=prompt),
            HumanMessage(content=f"ฉันอยู่ที่ {user_location_name}. ช่วยอธิบายเส้นทางไปยัง {destination_name} ให้หน่อย")
        ])
        if response and hasattr(response, 'content'):
            print(f"\nคำอธิบายเส้นทางไปยัง {destination_name}:\n{response.content}")
        else:
            print("ไม่พบคำตอบจาก LLM.")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดระหว่างเรียกใช้งาน LLM: {e}")

# ฟังก์ชันแสดง Dropdown สำหรับเลือกสถานที่
def display_places_list(places_from_api, user_location):
    if places_from_api and "data" in places_from_api:
        places = places_from_api["data"]
        options = [(place.get("name", "ไม่ทราบชื่อ"), (place.get("lat"), place.get("lon"))) for place in places]
        dropdown = widgets.Dropdown(
            options=options,
            description='สถานที่:'
        )

        def on_dropdown_change(change):
            selected_coords = change.new
            if selected_coords:
                dest_lat, dest_lon = selected_coords
                selected_place_name = dropdown.label
                print(f"\nคุณเลือกสถานที่: {selected_place_name}")
                print(f"ละติจูด: {dest_lat}, ลองจิจูด: {dest_lon}")
                print("\nกำลังดึงข้อมูลเส้นทาง...")
                route_info = get_route(user_location[0], user_location[1], dest_lat, dest_lon)
                if route_info:
                    display_route_map(route_info, user_location, (dest_lat, dest_lon))
                    explain_route_with_llm(route_info, selected_place_name, llm, user_location[0], user_location[1], longdo_api_key)
                else:
                    print("ไม่สามารถดึงข้อมูลเส้นทางได้")

        dropdown.observe(on_dropdown_change, names='value')
        display(dropdown)
    else:
        print("ไม่พบข้อมูลสถานที่ในผลลัพธ์")


# เอาตำแหน่งละติจูดและลองจิจูดเข้าไปเพื่อทำการถอดชื่อสภานที่ออกมา

https://api.longdo.com/map/services/address?  Reverse geocoding